In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from keras.applications import VGG16

In [13]:
conv_base_model = VGG16(weights='imagenet', input_shape=(150, 150, 3), include_top=False)

In [14]:
for layer in conv_base_model.layers:
    print(layer.name, layer.trainable)

input_3 False
block1_conv1 True
block1_conv2 True
block1_pool True
block2_conv1 True
block2_conv2 True
block2_pool True
block3_conv1 True
block3_conv2 True
block3_conv3 True
block3_pool True
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [17]:
# For some reason "conv_base_model.trainable = False" does not set "layer.trainable=False" for any layer
# workaround is in next cell

conv_base_model.trainable = False

for layer in conv_base_model.layers:
    print(layer.name, layer.trainable)

input_3 False
block1_conv1 True
block1_conv2 True
block1_pool True
block2_conv1 True
block2_conv2 True
block2_pool True
block3_conv1 True
block3_conv2 True
block3_conv3 True
block3_pool True
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [18]:
#for last 3 Conv and pooling layers, we want to set trainable == True

# iterate layers one by one, once you reach block5, start setting them as true

conv_base_model.trainable = False

TIME_TO_SET_FLAG = False

for layer in conv_base_model.layers:
    
    # bloack5 onwards, flip the flag
    if layer.name == 'block5_conv1':
        TIME_TO_SET_FLAG = True
    
    if TIME_TO_SET_FLAG == True:
        layer.trainable = True
    else:
        layer.trainable = False

In [19]:
for layer in conv_base_model.layers:
    print(layer.name, layer.trainable)

input_3 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [ ]:
# add model on top

In [22]:
from keras.models import Sequential
from keras import layers

In [23]:
model = Sequential()
model.add(conv_base_model)

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 513       
Total params: 18,910,017
Trainable params: 4,195,329
Non-trainable params: 14,714,688
_________________________________________________________________


### SetUp Data sources

In [25]:
import os
import numpy as np

In [26]:
pwd_ = os.getcwd()
pwd_

'D:\\Github\\DeepNets\\Keras\\Keras_from_scratch\\Vision'

In [27]:
path =  os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(pwd_)))))
print(path)

D:\


In [28]:
dest_data_folder = os.path.join(os.path.join(path, "Warehouse"), "cats_and_dogs")

train_dir = os.path.join(dest_data_folder, "train")
cat_train_dir = os.path.join(train_dir, "cat")
dog_train_dir = os.path.join(train_dir, "dog")


test_dir = os.path.join(dest_data_folder, "test")
cat_test_dir = os.path.join(test_dir, "cat")
dog_test_dir = os.path.join(test_dir, "dog")


validation_dir = os.path.join(dest_data_folder, "validation")
cat_validation_dir = os.path.join(validation_dir, "cat")
dog_validation_dir = os.path.join(validation_dir, "dog")

In [29]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 4000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=70,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/10
70/70 [==============================] - 899s 13s/step - loss: 0.5834 - acc: 0.7171 - val_loss: 0.4800 - val_acc: 0.8180
Epoch 2/10
65/70 [==========================>...] - ETA: 36s - loss: 0.4508 - acc: 0.8115